In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#database connection settings
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

In [3]:
#############################  UTILITIES ###############################

import re
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_en = set(stopwords.words('english'))
stop_de = set(stopwords.words('german'))
stop_it = set(stopwords.words('italian'))

def pre_process(text):  
    try:
        text = text.lower()    
        tokens = word_tokenize(text)
        tokens = [t for t in tokens if t .isalpha()]    
        tokens = [t for t in tokens if t not in stop_en]
        tokens = [t for t in tokens if t not in stop_de]
        tokens = [t for t in tokens if t not in stop_it]
    except:
        tokens=[]
    return tokens

def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    row_accumulator = []

    def splitListToRows(row, separator):
        split_row = str(row[target_column]).split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)

    df.apply(splitListToRows, axis=1, args = (separator, ))
    new_df = pd.DataFrame(row_accumulator)
    return new_df

##############################################################################################

In [24]:
######################  USE AKAS FOR WORKS BASED ON PREVIOUS MAPPINGS ########################
import pandas as pd

############################ database connection settings ####################################
import psycopg2
db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"
conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)
##############################################################################################

cursor = conn.cursor()  
query = "select name, work_type, uri, " + \
        "case when aka2 IS NULL then aka " + \
        "else aka || '; ' || aka2 end as aka " + \
        "from " + \
        "  (select name, aka, work_type, uri, string_agg(work,'; ') as aka2 " + \
        "  from " + \
        "    (SELECT DISTINCT w.name, w.aka, w.work_type, w.uri, lm.work " + \
        "     FROM works w " + \
        "       LEFT JOIN listing_mappers lm ON w.uri = lm.work_uri " + \
        "     WHERE w.uri != 'W1' " + \
        "     ORDER BY w.name " + \
        "    ) w " + \
        "  group by name, aka, work_type, uri) w"
cursor.execute(query)

w_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'work_type', 'uri', 'aka'])
# w_df = splitDataFrameList(w_df, 'aka', '; ')
w_df = w_df.drop_duplicates()

cursor = conn.cursor()  
query = "SELECT lower(l.composer) as composer, lower(l.work) as work, l.map_key, " + \
        "SUM(CASE WHEN l.source_id IN (6, 11, 14) THEN CAST(l.additional_text AS INT) ELSE 1 END) as listings " + \
        "FROM listings l " + \
        "LEFT JOIN listing_mappers lm ON l.map_key = lm.map_key " + \
        "WHERE lm.work_uri IS NULL and l.source_id = 5 " + \
        "GROUP BY l.composer, l.work, l.map_key "
cursor.execute(query)
l_df = pd.DataFrame(cursor.fetchall(), columns=['composer', 'work', 'map_key', 'listings'])

################################ MAP TOKENS CONTAINED IN LISTINGS ##############################

import Levenshtein


map_dict = {}
work_dict = {}

for idx2,w in w_df.iterrows():        
    work_tokens = pre_process(w['aka'])
    work_dict[w['uri']] = work_tokens

for idx,row in l_df.iterrows():
    
    if idx%250==0:
        print(idx)
        
    comp_key = row['composer']+'---'+row['work']
    tokens = pre_process(row['work'])        
    map_dict[comp_key] = {}
    
    for key in work_dict.keys():
        work_tokens = work_dict.get(key)
        if work_tokens:
            intersection = [value for value in tokens if value in work_tokens]
            counter = len(intersection)
            map_dict[comp_key][key] = [counter, counter/max(1, len(tokens))]
################################################################################################  

########################### FIND MATCHES BASED ON HIGHEST % CONTAINED ##########################

closest_works={}

for idx,key in enumerate(list(map_dict.keys())):
            
    w_dict = map_dict.get(key)
    if w_dict.keys():
        key_min = max(w_dict.keys(), key=(lambda k: w_dict[k]))
        dist = w_dict.get(key_min)[0]
        norm_dist = w_dict.get(key_min)[1]
        closest_works[key] = [key_min, dist, norm_dist]
    
cursor = conn.cursor()  
query = "SELECT w.name as name, w.uri, w.aka, w.work_type, c.name FROM works w " + \
        "LEFT JOIN composers c ON w.composer_id = c.id"
cursor.execute(query)
works_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'uri', 'aka', 'work_type', 'w_composer'])
cursor.execute(query)

df = pd.DataFrame.from_dict(closest_works, orient='index', columns=['pm_work_uri', 'pm_contained', 'pm_contained_perc'])
df['pm_work_composer'] = df.index
df = df.merge(works_df, left_on=['pm_work_uri'], right_on=['uri'], how='inner')
df = df[['pm_work_uri', 'pm_contained', 'pm_contained_perc', 'pm_work_composer', 'name','uri', 'w_composer']]

df = df[df['pm_contained']!=0].sort_values(by=['pm_contained_perc', 'uri'], ascending=False)
# df = df[df['pm_contained_perc']==1]
df = df[df['pm_contained_perc']>=0.5]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.8)]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.7) & (df['pm_contained_perc']<0.8)]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.6) & (df['pm_contained_perc']<0.7)]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.5) & (df['pm_contained_perc']<0.6)]
################################################################################################  

###################################### DEALING WITH MAPPINGS #########################################
import urllib.parse

def get_composer(text):
    return text.split('---')[0]
def get_work(text):
    return text.split('---')[1]
def url_text(text):
    return urllib.parse.quote_plus(text)

local_url = '<a href="http://localhost:3000/link_from_df?'

df['composer']=df.apply(lambda row: get_composer(row['pm_work_composer']), axis=1)
df['work']=df.apply(lambda row: get_work(row['pm_work_composer']), axis=1)
df['map'] = df.apply(lambda row: local_url + 'composer={0}&work={1}&work_uri={2}&source_id=9">link</a>'.format(url_text(row['composer']), url_text(row['work']), row['uri']), axis=1)
pd.set_option('display.max_colwidth', -1)
df = df[['pm_work_uri', 'pm_contained', 'pm_contained_perc', 'pm_work_composer', 'name', 'uri', 'w_composer', 'map']]

HTML(df.to_html(escape=False))

##############################################################################################            

0


,pm_work_uri,pm_contained,pm_contained_perc,pm_work_composer,name,uri,w_composer,map
228,W21923,1,1.000000,sally beamish---monster,The Monster in the Maze,W21923,Jonathan Dove,link
208,W21916,2,1.000000,michel-richard de lalande---te deum,Te Deum,W21916,Jean-Baptiste Lully,link
72,W21879,1,1.000000,antonio lotti---magnificat,Magnificat,W21879,Johann Christian Bach,link
73,W21879,1,1.000000,henry purcell---magnificat,Magnificat,W21879,Johann Christian Bach,link
237,W21785,3,1.000000,thomas morley---petite messe solennelle,La petite messe solennelle,W21785,Gioachino Rossini,link
195,Q94737,2,1.000000,louis mander---the life to come - an opera,A Life for the Tsar,Q94737,Mikhail Glinka,link
107,Q7823801,1,1.000000,colin riley---close,Too Close to the Sun,Q7823801,None,link
163,Q7752429,2,1.000000,"henry purcell---not to us, o lord",The Mother of Us All,Q7752429,Virgil Thomson,link
116,Q5518870,1,1.000000,ermanno wolf-ferrari---rita,Gallantry,Q5518870,None,link
93,Q54996248,1,1.000000,ben bernstein---the man in the mirror,A Mirror for the Sky,Q54996248,Gail Kubik,link


In [22]:
#############################  WORK NAMES CONTAINED IN LISTINGS ###############################
import pandas as pd

############################ database connection settings ####################################
import psycopg2
db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"
conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)
##############################################################################################


cursor = conn.cursor()  
query = "SELECT DISTINCT w.name, w.work_type, w.uri FROM works w"
cursor.execute(query)

w_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'work_type', 'uri'])
w_df = w_df.drop_duplicates()

cursor = conn.cursor()  
query = "SELECT lower(l.composer) as composer, lower(l.work) as work, l.map_key, " + \
        "SUM(CASE WHEN l.source_id IN (6, 11, 14) THEN CAST(l.additional_text AS INT) ELSE 1 END) as listings " + \
        "FROM listings l " + \
        "LEFT JOIN listing_mappers lm ON l.map_key = lm.map_key " + \
        "WHERE lm.work_uri IS NULL and l.source_id = 5 " + \
        "GROUP BY l.composer, l.work, l.map_key "
cursor.execute(query)
l_df = pd.DataFrame(cursor.fetchall(), columns=['composer', 'work', 'map_key', 'listings'])

################################ MAP TOKENS CONTAINED IN LISTINGS ##############################

import Levenshtein

map_dict = {}
work_dict = {}
list_dict = {}

for idx2,w in w_df.iterrows():        
    work_tokens = pre_process(w['name'])
    work_dict[w['uri']] = work_tokens

for idx2,l in l_df.iterrows(): 
    comp_key = l['composer']+'---'+l['work']
    listing_tokens_tokens = pre_process(l['work'])
    list_dict[comp_key] = [listing_tokens_tokens, l['listings']]
    
print('Total records: ' + str(len(work_dict)))

for idx,key in enumerate(work_dict):
    if idx%500==0:
        print(idx)

    map_dict[key] = {}        
    work_tokens = work_dict.get(key)
    
    for l_key in list_dict.keys():
        list_tokens = list_dict.get(l_key)[0]
        num_listings = list_dict.get(l_key)[1]
        if list_tokens:
            intersection = [value for value in work_tokens if value in list_tokens]
            counter = len(intersection)
            map_dict[key][l_key] = [counter, counter/max(1, len(work_tokens)), num_listings]                
            
################################################################################################  

########################### FIND MATCHES BASED ON HIGHEST % CONTAINED ##########################

closest_works={}

for idx,key in enumerate(list(map_dict.keys())):
            
    w_dict = map_dict.get(key)
    if w_dict.keys():
        key_min = max(w_dict.keys(), key=(lambda k: w_dict[k]))
        dist = w_dict.get(key_min)[0]
        norm_dist = w_dict.get(key_min)[1]
        num_listings = w_dict.get(key_min)[2]
        closest_works[key] = [key_min, dist, norm_dist, num_listings]
    
cursor = conn.cursor()  
query = "SELECT w.name as name, w.uri, w.aka, w.work_type, c.name FROM works w " + \
        "LEFT JOIN composers c ON w.composer_id = c.id"
cursor.execute(query)
works_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'uri', 'aka', 'work_type', 'w_composer'])

df = pd.DataFrame.from_dict(closest_works, orient='index', columns=['pm_work_composer', 'pm_contained', 'pm_contained_perc', 'pm_num_listings'])
df['pm_work_uri'] = df.index


df = df.merge(works_df, left_on=['pm_work_uri'], right_on=['uri'], how='inner')
df = df[['pm_work_uri', 'pm_contained', 'pm_contained_perc', 'pm_num_listings', 'pm_work_composer', 'name','uri','w_composer']]

df = df[df['pm_contained']!=0].sort_values(by=['pm_contained_perc', 'pm_num_listings', 'pm_work_composer', 'uri', 'w_composer'], ascending=False)

# df = df[(df['pm_num_listings'] < 2) & (df['pm_num_listings'] >= 1)]
df = df[df['pm_contained_perc']==1]
# df = df[df['pm_contained_perc']>=0.7]

# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.7)]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.7) & (df['pm_contained_perc']<0.8)]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.6) & (df['pm_contained_perc']<0.7)]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.5) & (df['pm_contained_perc']<0.6)]


###################################################################################################### 

###################################### DEALING WITH MAPPINGS #########################################

import urllib.parse

def get_composer(text):
    return text.split('---')[0]
def get_work(text):
    return text.split('---')[1]
def url_text(text):
    return urllib.parse.quote_plus(text)

local_url = '<a href="http://localhost:3000/link_from_df?'

df['composer']=df.apply(lambda row: get_composer(row['pm_work_composer']), axis=1)
df['work']=df.apply(lambda row: get_work(row['pm_work_composer']), axis=1)
df['map'] = df.apply(lambda row: local_url + 'composer={0}&work={1}&work_uri={2}&source_id=9">link</a>'.format(url_text(row['composer']), url_text(row['work']), row['uri']), axis=1)
pd.set_option('display.max_colwidth', -1)

df = df[['pm_contained', 'pm_contained_perc', 'pm_num_listings', 'pm_work_composer', 'name', 'w_composer', 'map']]

HTML(df.to_html(escape=False))      

###################################################################################################### 

Total records: 11652
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500


,pm_contained,pm_contained_perc,pm_num_listings,pm_work_composer,name,w_composer,map
5215,1,1.0,5,sir arthur sullivan---excerpts from 'the mikado',The Mikado,Arthur Sullivan,link
1070,1,1.0,5,louis mander---the life to come - an opera,After Life,Michel van der Aa,link
11327,1,1.0,5,louis mander---the life to come - an opera,After Life,Tom Cipullo,link
9401,1,1.0,5,louis mander---the life to come - an opera,The Life,Cy Coleman,link
7917,1,1.0,5,louis mander---the life to come - an opera,The Four-Note Opera,Tom Johnson,link
6001,1,1.0,5,louis mander---the life to come - an opera,L'Ouragan (opera),Alfred Bruneau,link
10866,1,1.0,5,louis mander---the life to come - an opera,Opera,Luciano Berio,link
7552,1,1.0,5,louis mander---the life to come - an opera,Akan-sere (opera),Sydyk Mukhamedzjanov,link
9255,1,1.0,4,matt & rob vale---nine days they fell,Nine,Maury Yeston,link
7821,1,1.0,3,samuel hogarth---david & goliath,David,Darius Milhaud,link


In [21]:
######################## MATCH ONLY LISTINGS CONTAINED IN WORK NAMES #########################
import pandas as pd

############################ database connection settings ####################################
import psycopg2
db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"
conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)
##############################################################################################


cursor = conn.cursor()  
query = "SELECT DISTINCT w.name, w.work_type, w.uri FROM works w"
cursor.execute(query)

w_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'work_type', 'uri'])
w_df = w_df.drop_duplicates()

cursor = conn.cursor()  
query = "SELECT lower(l.composer) as composer, lower(l.work) as work, l.map_key, " + \
        "SUM(CASE WHEN l.source_id IN (6, 11, 14) THEN CAST(l.additional_text AS INT) ELSE 1 END) as listings " + \
        "FROM listings l " + \
        "LEFT JOIN listing_mappers lm ON l.map_key = lm.map_key " + \
        "WHERE lm.work_uri IS NULL and l.source_id = 5 " + \
        "GROUP BY l.composer, l.work, l.map_key "
cursor.execute(query)
l_df = pd.DataFrame(cursor.fetchall(), columns=['composer', 'work', 'map_key', 'listings'])

################################ MAP TOKENS CONTAINED IN LISTINGS ##############################
import Levenshtein

map_dict = {}
work_dict = {}
list_dict = {}

for idx2,w in w_df.iterrows():        
    work_tokens = pre_process(w['name'])
    work_dict[w['uri']] = work_tokens

for idx2,l in l_df.iterrows(): 
    comp_key = l['composer']+'---'+l['work']
    listing_tokens = pre_process(l['work'])
    list_dict[comp_key] = [listing_tokens, l['listings']]
    
print('Total records: ' + str(len(list_dict)))
    
    
for idx,l_key in enumerate(list_dict):
    if idx%500==0:
        print(idx)

    map_dict[l_key] = {}        
    list_tokens = list_dict.get(l_key)[0]
    num_listings = list_dict.get(l_key)[1]

    for w_key in work_dict.keys():
        work_tokens = work_dict.get(w_key)
        if work_tokens:
            intersection = [value for value in work_tokens if value in list_tokens]
            counter = len(intersection)
            map_dict[l_key][w_key] = [counter, counter/max(1, len(list_tokens)), num_listings]                    
################################################################################################  

########################### FIND MATCHES BASED ON HIGHEST % CONTAINED ##########################

closest_works={}

for idx,key in enumerate(list(map_dict.keys())):
            
    l_dict = map_dict.get(key)
    if l_dict.keys():
        key_min = max(l_dict.keys(), key=(lambda k: l_dict[k]))
        dist = l_dict.get(key_min)[0]
        norm_dist = l_dict.get(key_min)[1]
        num_listings = l_dict.get(key_min)[2]
        closest_works[key] = [key_min, dist, norm_dist, num_listings]
    
cursor = conn.cursor()  
query = "SELECT w.name as name, w.uri, w.aka, w.work_type, c.name FROM works w LEFT JOIN composers c ON w.composer_id = c.id"
cursor.execute(query)
works_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'uri', 'aka', 'work_type', 'w_composer'])

df = pd.DataFrame.from_dict(closest_works, orient='index', columns=['pm_work_uri', 'pm_contained', 'pm_contained_perc', 'pm_num_listings'])
df['pm_work_composer'] = df.index
df = df.merge(works_df, left_on=['pm_work_uri'], right_on=['uri'], how='inner')
df = df[['pm_work_uri', 'pm_contained', 'pm_contained_perc', 'pm_num_listings', 'pm_work_composer', 'name','uri','w_composer']]

df = df[df['pm_contained']!=0].sort_values(by=['pm_num_listings', 'pm_contained_perc', 'pm_work_composer', 'uri', 'w_composer'], ascending=False)

#Resetting for memory savings
map_dict = {}
work_dict = {}
list_dict = {}
closest_works = {}

df = df[(df['pm_num_listings'] < 5) & (df['pm_num_listings'] >= 3)]
df = df[df['pm_contained_perc']>=1]
# df = df[df['pm_contained_perc']>=0.7]

# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.7)]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.7) & (df['pm_contained_perc']<0.8)]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.6) & (df['pm_contained_perc']<0.7)]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.5) & (df['pm_contained_perc']<0.6)]

################################################################################################  

###################################### DEALING WITH MAPPINGS #########################################
import urllib.parse

def get_composer(text):
    return text.split('---')[0]
def get_work(text):
    return text.split('---')[1]
def url_text(text):
    return urllib.parse.quote_plus(text)

local_url = '<a href="http://localhost:3000/link_from_df?'

df['composer']=df.apply(lambda row: get_composer(row['pm_work_composer']), axis=1)
df['work']=df.apply(lambda row: get_work(row['pm_work_composer']), axis=1)
df['map'] = df.apply(lambda row: local_url + 'composer={0}&work={1}&work_uri={2}&source_id=9">link</a>'.format(url_text(row['composer']), url_text(row['work']), row['uri']), axis=1)
pd.set_option('display.max_colwidth', -1)

df = df[['pm_contained', 'pm_contained_perc', 'pm_num_listings', 'pm_work_composer', 'name', 'w_composer', 'map']]
HTML(df.to_html(escape=False))
#########################################################################################################  

Total records: 285
0


,pm_contained,pm_contained_perc,pm_num_listings,pm_work_composer,name,w_composer,map
255,1,1.0,3,robert hugill---when a man knows,The Me Nobody Knows,Gary William Friedman,link


In [6]:
#################################  GENERATE UNKNOWN WORKS MAPPINGS ################################
import urllib.parse
import pandas as pd

def url_text(text):
    return urllib.parse.quote_plus(text)

cursor = conn.cursor()  
query = "SELECT lower(l.composer) as composer, lower(l.work) as work, l.map_key, " + \
        "SUM(CASE WHEN l.source_id IN (6, 11, 14) THEN CAST(l.additional_text AS INT) ELSE 1 END) as listings " + \
        "FROM listings l " + \
        "LEFT JOIN listing_mappers lm ON l.map_key = lm.map_key " + \
        "WHERE lm.work_uri IS NULL and l.source_id != 14 " + \
        "GROUP BY l.composer, l.work, l.map_key "
cursor.execute(query)
l_df = pd.DataFrame(cursor.fetchall(), columns=['composer', 'work', 'map_key', 'listings'])
local_url = '<a href="http://localhost:3000/link_from_df?'
l_df = l_df[l_df['work'].str.contains('operetta')]
l_df['map'] = l_df.apply(lambda row: local_url + 'composer={0}&work={1}&work_uri=W1&source_id=9">link</a>'.format(url_text(row['composer']), url_text(row['work'])), axis=1)
pd.set_option('display.max_colwidth', -1)
l_df = l_df.sort_values(by=['listings'], ascending=False)
HTML(l_df.to_html(escape=False))

##############################################################################################

,composer,work,map_key,listings,map
2965,,"operetta, my dear watson","operetta, my dear watson",10,link
2964,,operetta king imre kalman,operetta king imre kalman,6,link


In [ ]:
for idx,row in df.iterrows():
    composer = row['pm_work_composer'].split('---')[0]    
    work = row['pm_work_composer'].split('---')[1]
    map_key = composer+work
    insert = 'INSERT INTO LISTING_MAPPERS (WORK, COMPOSER, MAP_KEY, WORK_URI) VALUES ('
    values = "'" + work.replace("'", "''") + "', '" +  composer.replace("'", "''") + "', '" + map_key.replace("'", "''") + "', '" + row['pm_work_uri'] + "'); "
    commit = 'COMMIT;'    
    print(insert + values + commit)